<a href="https://colab.research.google.com/github/drummanbr/My_Second_LLM/blob/main/Imersao_IA_Aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [32]:
# Import the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
                          "\n"
                          "Artigo completo: \n"
                          "\n"
                          "Gemini API & Google Al Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                content=sample_text,

                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embeddings)


{'embedding': [0.06589348, -0.024026815, -0.022506025, 0.03657483, 0.077674635, -0.0020949352, -0.03309487, -0.022468025, 0.05813049, 0.058358967, 0.008831239, 0.023748437, -0.050900448, -0.022957956, 0.0023263278, -0.017176753, 0.022028355, -0.015392642, -0.026894422, -0.016287923, 0.014163538, 0.008585203, -0.048974328, -0.06085103, -0.026845958, 0.028030016, 0.019542975, -0.031202063, -0.024851259, 0.023163423, -0.056588393, 0.05719276, -0.039066177, 0.02031982, -0.034007534, -0.05284921, -0.02717991, -0.041481994, -0.0063827704, -0.003209544, 0.002941179, -0.09586273, -0.005349286, 0.01859883, 0.0007101435, -0.020257289, 0.04923351, 0.041000105, 0.0055106846, -0.06848173, 0.026700832, 0.020914456, 0.05886589, -0.045683235, -0.010544652, -0.0010499323, 0.017609838, -0.040673874, 0.013953233, -0.014915876, 0.005322981, 0.019404743, 0.0026244563, 0.053797584, 0.024847409, -0.06664162, -0.04865379, 0.011659653, 0.014663812, 0.04852437, 0.0048841573, 0.009225796, 0.055340596, -0.0487559

In [8]:
DOCUMENT1 = {
    "title": "Operação do Sistema de Controle de Climatização",
    "content": "Seu Googlecar possui um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar dentro do carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central."}
DOCUMENT2 = {
    "title": "Tela sensível ao toque",
    "content": "O Googlecar possui uma grande tela sensível ao toque que fornece acesso a diversos recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone \"Navegação\" para obter instruções para o seu destino ou tocar no ícone \"Música\" para tocar suas músicas favoritas."}
DOCUMENT3 = {
    "title": "Troca de Marchas",
    "content": "Seu Googlecar possui transmissão automática. Para trocar de marchas, basta mover a alavanca de câmbio para a posição desejada. Park (Estacionamento): Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverse (Ré): Esta posição é usada para dar ré. Neutral (Neutro): Esta posição é usada quando você está parado em um sinal ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive (Condução): Esta posição é usada para dirigir para frente. Low (Baixa): Esta posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Operação do Sistema de Controle de Climatização,Seu Googlecar possui um sistema de controle cl...
1,Tela sensível ao toque,O Googlecar possui uma grande tela sensível ao...
2,Troca de Marchas,Seu Googlecar possui transmissão automática. P...


In [11]:
model="models/embedding-001"

In [14]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                content=text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [15]:
df["embeddings"] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,embeddings
0,Operação do Sistema de Controle de Climatização,Seu Googlecar possui um sistema de controle cl...,"[-0.013456265, -0.026958406, -0.033347525, 0.0..."
1,Tela sensível ao toque,O Googlecar possui uma grande tela sensível ao...,"[0.027916193, -0.03778162, 0.016174262, 0.0058..."
2,Troca de Marchas,Seu Googlecar possui transmissão automática. P...,"[-0.0040780576, -0.022597747, -0.0064782393, 0..."


In [16]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Text"]

In [27]:
consulta = "Como se troca a marcha do googlecar?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar possui transmissão automática. Para trocar de marchas, basta mover a alavanca de câmbio para a posição desejada. Park (Estacionamento): Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Reverse (Ré): Esta posição é usada para dar ré. Neutral (Neutro): Esta posição é usada quando você está parado em um sinal ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Drive (Condução): Esta posição é usada para dirigir para frente. Low (Baixa): Esta posição é usada para dirigir na neve ou em outras condições escorregadias.


In [28]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [31]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Seu Googlecar é automático, parça!**

Tá ligado, pra trocar de marcha, é só dar um toque na alavanca e escolher o que você quer:

* **P (Parado):** Quando você tá estacionadão, as rodas ficam travadas igualzinho um cadeado.
* **R (Ré):** Pra dar aquela ré braba.
* **N (Neutro):** Quando você tá parado no farol ou no trânsito, o carro fica desligado e não anda nem que você pise no acelerador.
* **D (Dirigindo):** Pra ir pra frente, claro.
* **L (Baixa):** Pra quando você tá na neve ou no chão escorregadio, aí ele fica mais devagarinho.
